# simplelife Space Overview

This notebook tutorial explains the basics of spaces, by taking a closer look at spaces defined in the simplelife model as an example.

Click the badge below to run this notebook online on Google Colab. You need a Google account and need to be logged in to it to run this notebook on Google Colab.
[![Run on Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lifelib-dev/lifelib/blob/current/lifelib/projects/simplelife/simplelife-space-overview.ipynb)


The next code cell below is relevant only when you run this notebook on Google Colab. It installs lifelib and creates a copy of the library for this notebook.

In [ ]:
import sys, os

if 'google.colab' in sys.modules:
    lib = 'simplelife'; lib_dir = '/content/'+ lib
    if not os.path.exists(lib_dir):
        !pip install lifelib
        import lifelib; lifelib.create(lib, lib_dir)
        
    %cd $lib_dir

## Building a simplelife model

A new simplelife model is created and returned by ``build`` function in lifelib project. Import simplelife in your working folder.

Nextime you create the same model, you can give ``load_saved`` parameter ``True``, to save time to load input data.

In [1]:
import modelx as mx
model = mx.read_model("model")

The previously created model is renamed automatically to avoid name conflict. To get all existing models, ``get_models`` modelx API function can be used. ``get_models`` returns a dict of all the existing models associated with their names.

In [2]:
import modelx as mx
mx.get_models()

{'simplelife': <Model simplelife>}

## Access spaces in the model

In the model, there are child spaces and other objects referenced in the model. ``spaces`` property holds pairs of child space names and objects as a dict-like view.

In [3]:
model.spaces

{'Input': <UserSpace simplelife.Input>, 'LifeTable': <UserSpace simplelife.LifeTable>, 'Economic': <UserSpace simplelife.Economic>, 'BaseProj': <UserSpace simplelife.BaseProj>, 'PV': <UserSpace simplelife.PV>, 'Projection': <UserSpace simplelife.Projection>}

To just get all the names of the spaces, use the idiomatic expression below. To get all the space objects, ``values`` method can be used instead of ``keys``.  

In [4]:
list(model.spaces.keys())

['Input', 'LifeTable', 'Economic', 'BaseProj', 'PV', 'Projection']

To get a specific space, simply type its name as model's attribute, or pass the name as a key to spaces property. For example, to get ``Projection`` space:

In [5]:
model.Projection

<UserSpace simplelife.Projection>

or

In [6]:
model.spaces['Projection']

<UserSpace simplelife.Projection>

A space can in turn contain other spaces, forming a space tree. To access child spaces of a space, the ``spaces`` property and attribute access by name as we used on models can be used.

In [7]:
model.Projection.spaces

{'Policy': <UserSpace simplelife.Projection.Policy>, 'Assumptions': <UserSpace simplelife.Projection.Assumptions>}

In [8]:
model.Projection.Assumptions

<UserSpace simplelife.Projection.Assumptions>

In [9]:
model.Projection.spaces['Assumptions']

<UserSpace simplelife.Projection.Assumptions>

## Space overview

Below is the list of the names of the spaces in the simplelife model.

In [10]:
list(model.spaces.keys())

['Input', 'LifeTable', 'Economic', 'BaseProj', 'PV', 'Projection']

Below is a brief explanation of each space directly under the model.

* ``Input``: Parent space containing spaces and cells hoding data read from the input file.
* ``LifeTable``: Space containing cells related to commutation functions and actuarial notations.
* ``Policy``: Parametric space whose dynamic child spaces hold attribute data of each policy read from the input file. 
* ``Assumption``: Parametric space whose dynamic child spaces hold assumptions for each policy.
* ``Economic``: Parametric space whose dynamic child spaces hold economic assumptions for each scenario.
* ``BaseProj``: Base space of ``Projection`` space, which contains cells to carry out projections.
* ``PV``: Mixin space to ``Projection`` space, which contains cells to calculate present values of cashflows.
* ``Projection``: Parametric space, whose dynamic child spaces carry out projection for each policy.

## Base spaces

``BaseProj`` and ``PV`` are base spaces of ``Projection``, and cells defined in those cells are copied into ``Projection`` space.

In [11]:
model.Projection.bases

[<UserSpace simplelife.BaseProj>, <UserSpace simplelife.PV>]

## Parametric spaces

Among the spaces listed above, parametric spaces along with their parameters are listed below.

* ``LifeTable[Sex, IntRate, TableID]``
* ``Policy[PolicyID]``
* ``Assumption[PolicyID]``
* ``Economic[ScenID]``
* ``Projection[PolicyID, ScenID=1]``

To check parameters of a parametric space, call its ``parameters`` property.

In [12]:
model.LifeTable.parameters

('Sex', 'IntRate', 'TableID')

A space becomes parametric when it has the ``formula`` property. By calling the ``formula`` property, the source code of the function the formula is created from is printed.

The paramters of a parametric space, and their default values if any, are taken from the signature of its associated function.

In [13]:
model.LifeTable.formula

lambda Sex, IntRate, TableID: None

Parameter spaces serve as factories to create their child spaces dynamically.

When a parametric space is called with specific arguments being passed to the parameters, a dynamic child space associated with the arguments is created if it's not yet created, and returned. This can be achieved either by the call operation ``()`` or by subscription ``[]`` on the parametric space. If a parameter has its default value, and the arguments to the parameter is omitted, the default value is passed.

In [14]:
model.Projection[1]

<ItemSpace simplelife.Projection[1, 1]>

## Space formula

Space formulas have 2 roles relating to dynamic element spaces.

* To define the space's parameters and their default values. 
* To specify arguments used for constructing element spaces.

When the user tries to access the element space of a space, such as ``model.Projection[1]``, for the first time, the formula of the space is called in order to pass a dictionary of pairs of parameters and arguments used for constructing and initializing the element space.

The parameters include:

* ``bases``: a list of base spaces of the element space. If not specified, the space itself becomes the direct base space of the element space. 
* ``refs``: a dictionary of references to be defined in the element space.

Both of the parameters are optional.
As we have seen in the above, space formulas are created from function definitions.

In [15]:
model.Projection.formula

lambda PolicyID, ScenID=1: None

The formula code is executed in the namespace associated with the space. Note that the global scope of the function underlying the formula has nothing to do with the formula's scope. So, names such as ``Policy``, ``Assumption``, ``Economic`` that appear in the formula above are defined in the namespace of ``model.Projection``. 
The names defined in the namespace consists of child cells, child spaces and references in the space,  
i.e. the namespace is the union of ``cells``, ``spaces``, ``refs`` properties.

``dir`` function on a space lists all the names defined in the namespace associated with the space.

In [16]:
dir(model.Projection)

['AccumCF',
 'Assumptions',
 'AttAge',
 'BenefitAccDth',
 'BenefitAccHosp',
 'BenefitAnn',
 'BenefitDeath',
 'BenefitLiving',
 'BenefitMat',
 'BenefitOther',
 'BenefitSickHosp',
 'BenefitSurg',
 'BenefitSurr',
 'BenefitTotal',
 'ChangeRsrv',
 'DiscRate',
 'ExpsAcq',
 'ExpsAcqTotal',
 'ExpsCommInit',
 'ExpsCommRen',
 'ExpsCommTotal',
 'ExpsMaint',
 'ExpsMaintTotal',
 'ExpsOther',
 'ExpsTotal',
 'IncomeTotal',
 'InflFactor',
 'InsurIF_Beg1',
 'InsurIF_End',
 'IntAccumCF',
 'InterestNetCF',
 'InvstIncome',
 'InvstRetRate',
 'NetInsurCF',
 'PV_BenefitDeath',
 'PV_BenefitMat',
 'PV_BenefitSurr',
 'PV_BenefitTotal',
 'PV_Check',
 'PV_ExpsAcq',
 'PV_ExpsCommTotal',
 'PV_ExpsMaint',
 'PV_ExpsTotal',
 'PV_NetCashflow',
 'PV_NetCashflowForCheck',
 'PV_PremIncome',
 'PV_SumInsurIF',
 'Policy',
 'PolsAccDeath',
 'PolsAccHosp',
 'PolsAnnuity',
 'PolsDeath',
 'PolsIF_AftMat',
 'PolsIF_Beg',
 'PolsIF_Beg1',
 'PolsIF_End',
 'PolsLiving',
 'PolsMaturity',
 'PolsNewBiz',
 'PolsOther',
 'PolsRenewal',
 '